# Physics-Informed Neural Network

## Imports

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm

## Functions

In [4]:
class NN(nn.Module):
    def __init__(self, params):
        super(Network, self).__init__()
        
        # Unpack parameters
        self.n_in = params['n_in']
        self.n_hidden = params['n_hidden']
        self.n_out = params['n_out']
        
        # Define layers
        self.fully_connected1 = nn.Sequential(
            nn.Linear(self.n_in, self.n_hidden),
            nn.ReLU(),
            nn.Linear(self.n_hidden, self.n_hidden),
            nn.LeakyReLU(),
            )
        
        self.fully_connected2 = nn.Sequential(
            nn.Linear(self.n_hidden, self.n_out),
            )
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fully_connected1(x)
        x = self.fully_connected2(x)
        
        return x

In [ ]:
class PINN(nn.Module):
    def __init__(self, params):
        super(Network, self).__init__()
        
        # Unpack parameters
        self.n_in = params['n_in']
        self.n_hidden = params['n_hidden']
        self.n_out = params['n_out']
        
        # Define layers
        self.fully_connected1 = nn.Sequential(
            nn.Linear(self.n_in, self.n_hidden),
            nn.ReLU(),
            nn.Linear(self.n_hidden, self.n_hidden),
            nn.LeakyReLU(),
            )
        
        self.fully_connected2 = nn.Sequential(
            nn.Linear(self.n_hidden, self.n_out),
            )
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fully_connected1(x)
        x = self.fully_connected2(x)
        
        return x

In [ ]:
# Loss function
def loss_fun(output, target):
    return nn.MSELoss(output, target)

In [ ]:
def train(model, optimizer, num_epochs=10):
    
    out_dict = {'train_acc': [],
              'test_acc': [],
              'train_loss': [],
              'test_loss': []}
  
    for epoch in tqdm(range(num_epochs), unit='epoch'):
        
        # Activate train mode (beware with dropout)
        model.train()
        
        train_correct = 0
        train_loss = []
        
        for minibatch_no, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
            
            data, target = data.to(device), target.to(device)
            
            #Zero the gradients computed for each weight
            optimizer.zero_grad()
            
            #Forward pass your image through the network
            output = model(data)
            
            #Compute the loss
            loss = loss_fn(output, target)
            
            #Backward pass through the network
            loss.backward()
            
            #Update the weights
            optimizer.step()
            
            # Save train loss
            train_loss.append(loss.item())
            
            #Compute how many were correctly classified
            predicted = output.argmax(1)
            train_correct += (target==predicted).sum().cpu().item()
            
        #Compute the test accuracy
        test_loss = []
        test_correct = 0
        
        # Activate evaluation mode
        model.eval()
        
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            with torch.no_grad():
                output = model(data)
            test_loss.append(loss_fun(output, target).cpu().item())
            predicted = output.argmax(1)
            test_correct += (target==predicted).sum().cpu().item()
            
        out_dict['train_acc'].append(train_correct/len(trainset))
        out_dict['test_acc'].append(test_correct/len(testset))
        out_dict['train_loss'].append(np.mean(train_loss))
        out_dict['test_loss'].append(np.mean(test_loss))
        print(f"Loss train: {np.mean(train_loss):.3f}\t test: {np.mean(test_loss):.3f}\t",
              f"Accuracy train: {out_dict['train_acc'][-1]*100:.1f}%\t test: {out_dict['test_acc'][-1]*100:.1f}%")
    return out_dict

In [ ]:
def plot_results(results):
    plt.plot(range(10), results['test_loss'])
    plt.plot(range(10), results['train_loss'])
    plt.legend(('Test error', 'Train error'))
    plt.xlabel('Epoch number')
    plt.ylabel('Error')
    plt.show()

## Driver

In [9]:
# def main(params):

In [6]:
# Operations on the GPU if available
if torch.cuda.is_available():
    print("The code will run on GPU.")
else:
    print("The code will run on CPU.")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

The code will run on CPU.


In [ ]:
# Unpack parameters
lr = params['learning_rate']
n_epochs = params['n_epochs']
optim = params['optim']
loss_fun = params['loss_fun']

In [7]:
# Initialize model
model_nn = NN()
model_pinn = PINN()

# To GPU or CPU
model_nn.to(device)
model_pinn.to(device)

# Initialize the optimizer
if optim == 'SGD':
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
elif optim == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Set loss function
loss_fn = nn.MSELoss()

In [ ]:
# Train
results = train(model, optimizer, loss_fn, n_epochs)

## Data Visualization

In [ ]:
plot_results(results)

## .ipynb to .py

In [ ]:
# if __name__ == '__main__':
#     main()